#### LCEL (LangChain Expression Language) ແມ່ນຫຍັງ:
**LCEL ແມ່ນ syntax ຂອງ LangChain ທີ່ໃຊ້ໃນການເຊື່ອມຕໍ່ components ຕ່າງໆເຂົ້າດ້ວຍກັນ ແລະ ສາມາດເຮັດໃຫ້ອ່ານງ່າຍ. ໂດຍມັກຈະໃຊ້ເຄື່ອງໝາຍ | (pipe) ເພື່ອເຊື່ອມຕໍ່ components ຕ່າງໆໃນລຳດັບ.**

**HumanMessage:**
- ເປັນ class ສຳຫຼັບຈັດເກັບຂໍ້ຄວາມຂອງຜູ້ໃຊ້
- ແທນຄຳສັ່ງ ຫຼື ຄຳຖາມທີ່ User ສົ່ງມາ 

**SystemMessage:**
- ເປັນ class ສຳຫຼັບຈັດເກັບຄຳສັ່ງຂອງລະບົບ
- ກຳນົດບົດບາດ ແລະ ພຶດຕິກຳຂອງ AI
- ຕັ້ງກົດລະບຽບໃຫ້ AI ປະຕິບັດຕາມ

**StrOutputParser:**
- ເປັນ parser ທີ່ປ່ຽນຜົນລັບຈາກ AI model ໃຫ້ເປັນ string
- ເອົາແຕ່ເນື້ອໃນຂອງການຕອບກັບ
- ກຳຈັດ metadata ແລະ ຂໍ້ມູນພິເສດອື່ນໆ

**Chain:**
- ເປັນການເຊື່ອມຕໍ່ components ຫຼາຍໂຕເຂົ້າດ້ວຍກັນ
- ສາມາດເຮັດວຽກແບບລຳດັບຂັ້ນຕອນ
- ຜົນລັບຈາກ component ກ່ອນຈະເປັນ input ຂອງ component ຕໍ່ໄປ

**ChatPromptTemplate:**
- ເປັນ template ສຳຫຼັບສ້າງ prompts ສຳຫຼັບ chat models
- ສາມາດປະກອບຫຼາຍ message types ເຂົ້າດ້ວຍກັນ
- ມີ variables ທີ່ສາມາດປ່ຽນແປງຄ່າໄດ້
- ຊ່ວຍໃຫ້ການສ້າງ prompts ເປັນລະບົບ ແລະ ນຳໃຊ້ຄືນໄດ້ 

In [8]:
import os
from dotenv import load_dotenv
load_dotenv() 
groq_api_key=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model=ChatGroq(model="meta-llama/llama-4-maverick-17b-128e-instruct",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024460E26710>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024460E27110>, model_name='meta-llama/llama-4-maverick-17b-128e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Lao"),
    HumanMessage(content="Hello, How is going today ?")
]

result=model.invoke(messages)

In [12]:
result

AIMessage(content='ສະບາຍດີ ມື້ນີ້ເປັນແນວໃດ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 28, 'total_tokens': 48, 'completion_time': 0.097002696, 'prompt_time': 0.000814175, 'queue_time': 0.057989685, 'total_time': 0.097816871}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_c527aa4474', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bc33a2bd-3db1-4d71-a361-9985c9a969ea-0', usage_metadata={'input_tokens': 28, 'output_tokens': 20, 'total_tokens': 48})

In [13]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'ສະບາຍດີ ມື້ນີ້ເປັນແນວໃດ?'

In [14]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'ສະບາຍດີ ມື້ນີ້ເປັນແນວໃດ?'

In [15]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [16]:
result=prompt.invoke({"language":"Lao","text":"Hello, How is going today ?"})

In [17]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into Lao:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, How is going today ?', additional_kwargs={}, response_metadata={})]

In [18]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Lao","text":"Hello, How is going today ?"})

'ສະບາຍດີ, ມື້ນີ້ເປັນແນວໃດ?'